In [1]:
import pandas as pd 
import numpy as np

## Introduction

Each cell will have the solution to the problems

In [3]:
# 1. Download the event_data.csv dataset, which contains data on the use of the mobile
# application of users who registered from July 29 to September 1, 2019

df = pd.read_csv("../files/event_data.csv")
df.head()

,user_id,event_date,event_type,purchase_amount
0,c40e6a,2019-07-29 00:02:15,registration,NaN
1,a2b682,2019-07-29 00:04:46,registration,NaN
2,9ac888,2019-07-29 00:13:22,registration,NaN
3,93ff22,2019-07-29 00:16:47,registration,NaN
4,65ef85,2019-07-29 00:19:23,registration,NaN


In [5]:
# 2. Highlight user cohorts based on the week of registration in the application. The cohort
# identifier should be the week ordinal (for example, the week from July 29 to August 4
# should have identifier 31).

def week_cohort(user: pd.DataFrame):
    # Determine user cohort based on the week of registration in the application
    cohort = user[user['event_type']=='registration']
    cohort_week = cohort['event_date'].dt.isocalendar().week
    return cohort_week.iloc[0]

df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df['cohort'] = df.user_id.map(df.groupby(['user_id']).apply(week_cohort))

print(f"Check range date: {df.event_date.min()} - {df.event_date.max()}")
df.head(5)

Check range date: 2019-07-29 00:02:15 - 2019-09-01 23:59:54


,user_id,event_date,event_type,purchase_amount,cohort
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31
1,a2b682,2019-07-29 00:04:46,registration,NaN,31
2,9ac888,2019-07-29 00:13:22,registration,NaN,31
3,93ff22,2019-07-29 00:16:47,registration,NaN,31
4,65ef85,2019-07-29 00:19:23,registration,NaN,31


In [9]:
print("Check cohort 32")
df[df['cohort']==32]

Check cohort 32


,user_id,event_date,event_type,purchase_amount,cohort
5758,737199,2019-08-05 00:01:21,registration,NaN,32
5759,fc28ac,2019-08-05 00:01:29,registration,NaN,32
5774,11cc3f,2019-08-05 00:21:17,registration,NaN,32
5780,48f5ff,2019-08-05 00:27:14,registration,NaN,32
5790,c1c411,2019-08-05 00:39:10,registration,NaN,32
...,...,...,...,...,...
79723,1e19f1,2019-09-01 23:51:09,simple_event,NaN,32
79724,7f6003,2019-09-01 23:52:27,simple_event,NaN,32
79729,93f58b,2019-09-01 23:54:40,simple_event,NaN,32
79737,930c23,2019-09-01 23:57:41,simple_event,NaN,32


In [11]:
# 3. How many unique users in the cohort with ID 33?
print("Unique users cohort 33", df[df['cohort']==33]['user_id'].nunique())

Unique users cohort 33 2045


In [15]:
# 4. For each event, highlight the indicator lifetime - the weekly lifetime of the cohort. The
# lifetime indicator is calculated based on the serial number of the week in which the event
# is committed, relative to the week of registration. For example, an event committed on
# August 3 by a user from a cohort of registrants at 31 weeks will be committed on the zero
# week of lifetime, and an event committed by the same user on August 5 will be committed
# on the first week of lifetime).

df['week_date'] = df.event_date.dt.isocalendar().week
df['life_time_event'] = df['week_date'] - df['cohort']
df

,user_id,event_date,event_type,purchase_amount,cohort,week_date,life_time_event
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31,31,0
1,a2b682,2019-07-29 00:04:46,registration,NaN,31,31,0
2,9ac888,2019-07-29 00:13:22,registration,NaN,31,31,0
3,93ff22,2019-07-29 00:16:47,registration,NaN,31,31,0
4,65ef85,2019-07-29 00:19:23,registration,NaN,31,31,0
...,...,...,...,...,...,...,...
79737,930c23,2019-09-01 23:57:41,simple_event,NaN,32,35,3
79738,a84999,2019-09-01 23:57:50,simple_event,NaN,33,35,2
79739,175e4d,2019-09-01 23:59:40,simple_event,NaN,32,35,3
79740,1c2210,2019-09-01 23:59:51,simple_event,NaN,33,35,2


In [23]:
# 5. Build a summary table of changes in the Retention Rate for cohorts depending on lifetime.
groupby_table = df.groupby(['cohort','life_time_event'])['user_id'].nunique() / df.groupby(['cohort'])['user_id'].nunique()
groupby_table = groupby_table.reset_index().rename(columns={'user_id':'retention_rate'})
pivot_table = groupby_table.pivot_table(index='cohort', columns='life_time_event', values='retention_rate')
pivot_table = pivot_table.fillna(0)
pivot_table

life_time_event,0,1,2,3,4
cohort,,,,,
31,1.0,0.927595,0.629367,0.356962,0.15038
32,1.0,0.929303,0.648053,0.361168,0.00000
33,1.0,0.924205,0.661125,0.000000,0.00000
34,1.0,0.929078,0.000000,0.000000,0.00000
35,1.0,0.000000,0.000000,0.000000,0.00000


In [28]:
# 6. What is the 3 week retention rate for a cohort with ID 32? Give the answer in percent,
# rounded to 2 decimal places, inclusive.

value = groupby_table[(groupby_table['cohort']==32)&((groupby_table['life_time_event']==3))]['retention_rate'].iloc[0]
print(f"Retention rate week 3 and cohort ID 32: {np.round(value*100,2)}%")

Retention rate week 3 and cohort ID 32: 36.12%


In [39]:
# 7. Build a summary table of changes in the indicator ARPPU (Average Revenue Per Paying
# User) for cohorts depending on lifetime

purcharse_df = df[df['event_type']=='purchase'].copy()
arpu = lambda x: x['purchase_amount'].sum()/x['user_id'].nunique()
arpu_df = purcharse_df.groupby(['cohort','life_time_event']).apply(arpu).reset_index()
arpu_df.rename(columns={0:'arppu'}, inplace = True)
arpu_pivot_df = arpu_df.pivot_table(index='cohort', columns='life_time_event', values='arppu')
arpu_pivot_df

life_time_event,0,1,2,3,4
cohort,,,,,
31,30.445205,35.413793,35.911765,33.081081,31.40625
32,32.978723,35.024958,35.294118,33.297872,NaN
33,33.186441,35.858086,34.784810,NaN,NaN
34,33.843416,35.904936,NaN,NaN,NaN
35,34.094955,NaN,NaN,NaN,NaN


In [44]:
# 8. What is the 3-week ARPPU of a cohort with ID 31? Give the answer with a floating point
# number, rounded to 2 decimal places, inclusive.

value = arpu_df[(arpu_df['cohort']==31)&((arpu_df['life_time_event']==3))]['arppu'].iloc[0]
print("Retention rate week 3 and cohort ID 31:", np.round(value,2))

Retention rate week 3 and cohort ID 31: 33.08


In [46]:
# 9. What is the median time between user registration and first purchase? Give the answer
# in seconds (!) As an integer.

def time_date(user: pd.DataFrame):
    # Determine time between user registration and first purchase
    register_row = user[user['event_type']=='registration']
    purchase_row = user[user['event_type']=='purchase'].sort_values(by='event_date')
    if purchase_row.empty or register_row.empty:
        days_re_pu = np.nan
    else:
        days_re_pu = (purchase_row['event_date'].iloc[0]-register_row['event_date'].iloc[0])
    return days_re_pu

purcharse_date = df.groupby(['user_id']).apply(time_date).reset_index()
purcharse_date.rename(columns={0:'time_purcharse'}, inplace=True)
median_time_p = int(purcharse_date['time_purcharse'].median().total_seconds())
print(f"The median time between time and purcharse is {median_time_p} seconds")

The median time between time and purcharse is 434774 seconds
